Subiectul 1

In [ ]:
#a
import pandas as pd
import pymc as pm
import numpy as np
import arviz as az

data = pd.read_csv("/content/sample_data/date_alegeri_turul2.csv")

age = data['Varsta'].values
gender = data['Sex'].values
education = data['Educatie'].values
income = data['Venit'].values
vote = data['Vot'].values

with pm.Model() as model:
    beta_age = pm.Normal('beta_age', mu=0, sigma=10)
    beta_gender = pm.Normal('beta_gender', mu=0, sigma=10)
    beta_education = pm.Normal('beta_education', mu=0, sigma=10)
    beta_income = pm.Normal('beta_income', mu=0, sigma=10)
    intercept = pm.Normal('intercept', mu=0, sigma=10)

    logits = (
        beta_age * age +
        beta_gender * gender +
        beta_education * education +
        beta_income * income +
        intercept
    )

    p = pm.math.sigmoid(logits)
    likelihood = pm.Bernoulli('likelihood', p=p, observed=vote)

    trace = pm.sample(1000, tune=1000, return_inferencedata=True)

#b Variabilele cu cele mai mari valori absolute ale coeficienților (beta_*) sunt cele mai importante, deci influențează cel mai mult rezultatul.
az.plot_posterior(trace, var_names=["beta_age", "beta_gender", "beta_education", "beta_income"], hdi_prob=0.94)

#c
with pm.Model() as reduced_model:
    beta_age = pm.Normal('beta_age', mu=0, sigma=10)
    beta_income = pm.Normal('beta_income', mu=0, sigma=10)
    intercept = pm.Normal('intercept', mu=0, sigma=10)

    logits = beta_age * age + beta_income * income + intercept
    p = pm.math.sigmoid(logits)
    likelihood = pm.Bernoulli('likelihood', p=p, observed=vote)

    trace_reduced = pm.sample(1000, tune=1000, return_inferencedata=True)
    posterior_predictive = pm.sample_posterior_predictive(trace)

az.plot_posterior(trace_reduced, hdi_prob=0.94)

In [ ]:
#d
waic_full = az.waic(trace)
waic_reduced = az.waic(trace_reduced)

loo_full = az.loo(trace)
loo_reduced = az.loo(trace_reduced)

print("WAIC pt modelul complet:", waic_full)
print("WAIC pt modelul redus:", waic_reduced)

print("LOO pt modelul complet:", loo_full)
print("LOO pt modelul redus:", loo_reduced)

Subiectul 2

In [ ]:
#a
import pandas as pd

df = pd.read_csv('/content/sample_data/iris.csv')

print(df.head())
print(df.info())

df['species'] = df['species'].astype('category').cat.codes

X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y = df['species']

import pymc as pm
import numpy as np
import arviz as az
import matplotlib.pyplot as plt

with pm.Model() as model:
    n_groups = 3

    mu = pm.Normal('mu', mu=0, shape=(n_groups, X.shape[1]))
    sigma = pm.HalfNormal('sigma', shape=(n_groups, X.shape[1]))

    group = pm.Categorical('group', p=[1/n_groups] * n_groups, shape=len(X))

    Y_obs = pm.Normal('Y_obs', mu=mu[group], observed=X.values)

    trace = pm.sample(1000, return_inferencedata=False)
az.plot_trace(trace)
plt.show()

summary = pm.summary(trace)
print(summary)

#b
#Pentru a evalua ce caracteristică separă cel mai bine datele, trebuie analizată importanța acestora pe baza distribuției posteriori a coeficientilor modelului.
#Pot utiliza medii și intervale de încredere pentru fiecare caracteristică și analizez care caracteristică are cel mai mare impact asupra separării speciilor.

#Pe baza analizei distribuțiilor coeficientilor (medii și intervale de încredere), putem observa care caracteristică (sepal_length, sepal_width, petal_length, petal_width) contribuie cel mai mult la separarea speciilor. De exemplu, dacă petal_length are un coeficient cu o distribuție clar diferențiată față de celelalte, acest lucru sugerează că această caracteristică este cel mai important factor în clasificarea speciilor.

In [20]:
!pip install --upgrade arviz